<a href="https://colab.research.google.com/github/Baekhyunjung/study_nlp/blob/main/nlp_camp/study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchtext
version = list(map(int, torchtext.__version__.split('.')))
if version[0] <= 0 and version[1] < 9:
    from torchtext import data
else:
    from torchtext.legacy import data


class DataLoader(object):
    '''
    Data loader class to load text file using torchtext library.
    '''

    def __init__(
        self, train_fn,
        batch_size=64,
        valid_ratio=.2,
        device=-1,
        max_vocab=999999,
        min_freq=1,
        use_eos=False,
        shuffle=True,
    ):
        '''
        DataLoader initialization.
        :param train_fn: Train-set filename
        :param batch_size: Batchify data fot certain batch size.
        :param device: Device-id to load data (-1 for CPU)
        :param max_vocab: Maximum vocabulary size
        :param min_freq: Minimum frequency for loaded word.
        :param use_eos: If it is True, put <EOS> after every end of sentence.
        :param shuffle: If it is True, random shuffle the input data.
        '''
        super().__init__()

        # Define field of the input file.
        # The input file consists of two fields.
        self.label = data.Field(
            sequential=False,
            use_vocab=True,
            unk_token=None
        )
        self.text = data.Field(
            use_vocab=True,
            batch_first=True,
            include_lengths=False,
            eos_token='<EOS>' if use_eos else None,
        )

        # Those defined two columns will be delimited by TAB.
        # Thus, we use TabularDataset to load two columns in the input file.
        # We would have two separate input file: train_fn, valid_fn
        # Files consist of two columns: label field and text field.
        train, valid = data.TabularDataset(
            path=train_fn,
            format='tsv',
            fields=[
                ('label', self.label),
                ('text', self.text),
            ],
        ).split(split_ratio=(1 - valid_ratio))

        # Those loaded dataset would be feeded into each iterator:
        # train iterator and valid iterator.
        # We sort input sentences by length, to group similar lengths.
        self.train_loader, self.valid_loader = data.BucketIterator.splits(
            (train, valid),
            batch_size=batch_size,
            device='cuda:%d' % device if device >= 0 else 'cpu',
            shuffle=shuffle,
            sort_key=lambda x: len(x.text),
            sort_within_batch=True,
        )

        # At last, we make a vocabulary for label and text field.
        # It is making mapping table between words and indice.
        self.label.build_vocab(train)
        self.text.build_vocab(train, max_size=max_vocab, min_freq=min_freq)

In [ ]:
from torchtext import data, datasets
import torch.utils.data


PAD, BOS, EOS = 1, 2, 3


class DataLoader():

    def __init__(self,
                 train_fn,
                 valid_fn,
                 batch_size=64,
                 device='cpu',
                 max_vocab=99999999,
                 max_length=255,
                 fix_length=None,
                 use_bos=True,
                 use_eos=True,
                 shuffle=True
                 ):

        super(DataLoader, self).__init__()

        self.text = data.Field(sequential=True,
                               use_vocab=True,
                               batch_first=True,
                               include_lengths=True,
                               fix_length=fix_length,
                               init_token='<BOS>' if use_bos else None,
                               eos_token='<EOS>' if use_eos else None
                               )

        train = LanguageModelDataset(path=train_fn,
                                     fields=[('text', self.text)],
                                     max_length=max_length
                                     )
        valid = LanguageModelDataset(path=valid_fn,
                                     fields=[('text', self.text)],
                                     max_length=max_length
                                     )

        self.train_iter = data.BucketIterator(train,
                                              batch_size=batch_size,
                                              device='cuda:%d' % device if device >= 0 else 'cpu',
                                              shuffle=shuffle,
                                              sort_key=lambda x: -len(x.text),
                                              sort_within_batch=True
                                              )
        self.valid_iter = data.BucketIterator(valid,
                                              batch_size=batch_size,
                                              device='cuda:%d' % device if device >= 0 else 'cpu',
                                              shuffle=False,
                                              sort_key=lambda x: -len(x.text),
                                              sort_within_batch=True
                                              )

        self.text.build_vocab(train, max_size=max_vocab)


class LanguageModelDataset(data.Dataset):

    def __init__(self, path, fields, max_length=None, **kwargs):
        if not isinstance(fields[0], (tuple, list)):
            fields = [('text', fields[0])]

        examples = []
        with open(path) as f:
            for line in f:
                line = line.strip()
                if max_length and max_length < len(line.split()):
                    continue
                if line != '':
                    examples.append(data.Example.fromlist(
                        [line], fields))

        super(LanguageModelDataset, self).__init__(examples, fields, **kwargs)


if __name__ == '__main__':
    import sys
    loader = DataLoader(sys.argv[1], sys.argv[2])

    for batch_index, batch in enumerate(loader.train_iter):
        print(batch.text)

        if batch_index > 1:
            break

AttributeError: ignored

In [ ]:
import os

import torchtext
version = list(map(int, torchtext.__version__.split('.')))
if version[0] <= 0 and version[1] < 9:
    from torchtext import data
else:
    from torchtext.legacy import data

PAD, BOS, EOS = 1, 2, 3


class DataLoader():

    def __init__(self,
                 train_fn=None,
                 valid_fn=None,
                 exts=None,
                 batch_size=64,
                 device='cpu',
                 max_vocab=99999999,
                 max_length=255,
                 fix_length=None,
                 use_bos=True,
                 use_eos=True,
                 shuffle=True,
                 dsl=False
                 ):

        super(DataLoader, self).__init__()

        self.src = data.Field(
            sequential=True,
            use_vocab=True,
            batch_first=True,
            include_lengths=True,
            fix_length=fix_length,
            init_token='<BOS>' if dsl else None,
            eos_token='<EOS>' if dsl else None,
        )

        self.tgt = data.Field(
            sequential=True,
            use_vocab=True,
            batch_first=True,
            include_lengths=True,
            fix_length=fix_length,
            init_token='<BOS>' if use_bos else None,
            eos_token='<EOS>' if use_eos else None,
        )

        if train_fn is not None and valid_fn is not None and exts is not None:
            train = TranslationDataset(
                path=train_fn,
                exts=exts,
                fields=[('src', self.src), ('tgt', self.tgt)],
                max_length=max_length
            )
            valid = TranslationDataset(
                path=valid_fn,
                exts=exts,
                fields=[('src', self.src), ('tgt', self.tgt)],
                max_length=max_length,
            )

            self.train_iter = data.BucketIterator(
                train,
                batch_size=batch_size,
                device='cuda:%d' % device if device >= 0 else 'cpu',
                shuffle=shuffle,
                sort_key=lambda x: len(x.tgt) + (max_length * len(x.src)),
                sort_within_batch=True,
            )
            self.valid_iter = data.BucketIterator(
                valid,
                batch_size=batch_size,
                device='cuda:%d' % device if device >= 0 else 'cpu',
                shuffle=False,
                sort_key=lambda x: len(x.tgt) + (max_length * len(x.src)),
                sort_within_batch=True,
            )

            self.src.build_vocab(train, max_size=max_vocab)
            self.tgt.build_vocab(train, max_size=max_vocab)

    def load_vocab(self, src_vocab, tgt_vocab):
        self.src.vocab = src_vocab
        self.tgt.vocab = tgt_vocab


class TranslationDataset(data.Dataset):
    """Defines a dataset for machine translation."""

    @staticmethod
    def sort_key(ex):
        return data.interleave_keys(len(ex.src), len(ex.trg))

    def __init__(self, path, exts, fields, max_length=None, **kwargs):
        """Create a TranslationDataset given paths and fields.

        Arguments:
            path: Common prefix of paths to the data files for both languages.
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            Remaining keyword arguments: Passed to the constructor of
                data.Dataset.
        """
        if not isinstance(fields[0], (tuple, list)):
            fields = [('src', fields[0]), ('trg', fields[1])]

        if not path.endswith('.'):
            path += '.'

        src_path, trg_path = tuple(os.path.expanduser(path + x) for x in exts)

        examples = []
        with open(src_path, encoding='utf-8') as src_file, open(trg_path, encoding='utf-8') as trg_file:
            for src_line, trg_line in zip(src_file, trg_file):
                src_line, trg_line = src_line.strip(), trg_line.strip()
                if max_length and max_length < max(len(src_line.split()), len(trg_line.split())):
                    continue
                if src_line != '' and trg_line != '':
                    examples += [data.Example.fromlist([src_line, trg_line], fields)]

        super().__init__(examples, fields, **kwargs)


if __name__ == '__main__':
    import sys
    loader = DataLoader(
        sys.argv[1],
        sys.argv[2],
        (sys.argv[3], sys.argv[4]),
        batch_size=128
    )

    print(len(loader.src.vocab))
    print(len(loader.tgt.vocab))

    for batch_index, batch in enumerate(loader.train_iter):
        print(batch.src)
        print(batch.tgt)

        if batch_index > 1:
            break

IndexError: ignored

In [ ]:
import re, collections
from IPython.display import display, Markdown, Latex

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

num_merges = 10

dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))


print(bpe_codes)

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}
{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


In [ ]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # 특별 토큰인 </w>는 출력하지 않는다.
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)

    return word

In [ ]:
encode("loki")

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('l', 'o'), ('k', 'i'), ('o', 'k'), ('i', '</w>')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')


__Iteration 2:__

bigrams in the word: {('lo', 'k'), ('k', 'i'), ('i', '</w>')}
candidate for merging: ('lo', 'k')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [ ]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('o', 'w'), ('t', '</w>'), ('s', 't'), ('w', 'e'), ('e', 's'), ('l', 'o')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('o', 'w'), ('t', '</w>'), ('w', 'es'), ('es', 't'), ('l', 'o')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
candidate for merging: ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [ ]:
encode("lowing")

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('o', 'w'), ('n', 'g'), ('w', 'i'), ('i', 'n'), ('g', '</w>'), ('l', 'o')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')


__Iteration 2:__

bigrams in the word: {('n', 'g'), ('w', 'i'), ('i', 'n'), ('g', '</w>'), ('lo', 'w')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

bigrams in the word: {('i', 'n'), ('g', '</w>'), ('low', 'i'), ('n', 'g')}
candidate for merging: ('i', 'n')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [ ]:
encode("highing")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('g', 'h'), ('n', 'g'), ('i', 'n'), ('g', '</w>'), ('h', 'i'), ('i', 'g')}
candidate for merging: ('g', 'h')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'g')

In [ ]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')

29.5MB [00:00, 62.8MB/s]


In [ ]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [ ]:
next(train_iter)

(3,
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')

In [ ]:
next(train_iter)

(3,
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.")

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
vocab(['here', 'is', 'an', 'example'])

[475, 21, 30, 5297]

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [ ]:
text_pipeline('here is the an example')

[475, 21, 2, 30, 5297]

In [ ]:
label_pipeline('10')

9

In [ ]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

1.86MB [00:00, 16.1MB/s]


| epoch   1 |   500/ 1782 batches | accuracy    0.682
| epoch   1 |  1000/ 1782 batches | accuracy    0.852
| epoch   1 |  1500/ 1782 batches | accuracy    0.874
-----------------------------------------------------------
| end of epoch   1 | time: 14.73s | valid accuracy    0.896 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.900
| epoch   2 |  1000/ 1782 batches | accuracy    0.898
| epoch   2 |  1500/ 1782 batches | accuracy    0.903
-----------------------------------------------------------
| end of epoch   2 | time: 14.60s | valid accuracy    0.908 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.913
| epoch   3 |  1000/ 1782 batches | accuracy    0.914
| epoch   3 |  1500/ 1782 batches | accuracy    0.913
-----------------------------------------------------------
| end of epoch   3 | time: 14.77s | valid accuracy    0.913 
-------------------------------

In [ ]:
# 5장

from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

def hypernyms(word):
  current_node = wn.synsets(word)[0]
  yield current_node

  while True:
    try:
      current_node = current_node.hypernyms()[0]
      yield current_node
    except IndexError:
      break

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
wn.synsets('policeman')

[Synset('policeman.n.01')]

In [ ]:
for h in hypernyms('policeman'):
  print(h)

Synset('policeman.n.01')
Synset('lawman.n.01')
Synset('defender.n.01')
Synset('preserver.n.03')
Synset('person.n.01')
Synset('causal_agent.n.01')
Synset('physical_entity.n.01')
Synset('entity.n.01')


In [ ]:
for h in hypernyms('student'):
  print(h)

Synset('student.n.01')
Synset('enrollee.n.01')
Synset('person.n.01')
Synset('causal_agent.n.01')
Synset('physical_entity.n.01')
Synset('entity.n.01')


In [ ]:
policeman = wn.synset('policeman.n.01')
print(policeman.definition())

a member of a police force


In [ ]:
wn.synsets('man')

[Synset('man.n.01'),
 Synset('serviceman.n.01'),
 Synset('man.n.03'),
 Synset('homo.n.02'),
 Synset('man.n.05'),
 Synset('man.n.06'),
 Synset('valet.n.01'),
 Synset('man.n.08'),
 Synset('man.n.09'),
 Synset('man.n.10'),
 Synset('world.n.08'),
 Synset('man.v.01'),
 Synset('man.v.02')]

In [ ]:
man = wn.synset('man.n.01')
print(man.definition())

an adult person who is male (as opposed to a woman)


In [ ]:
import numpy as np

def distance(word1, word2):
    word1_hypernyms = [h for h in hypernyms(word1)]

    for i, word2_hypernym in enumerate(hypernyms(word2)):
        try:
            return i + word1_hypernyms.index(word2_hypernym)
        except ValueError:
            continue

print(distance('policeman', 'student'))

def similarity(word1, word2):
    return -np.log(distance(word1, word2))

print(similarity('policeman', 'student'))

6
-1.791759469228055


In [ ]:
print(man.hypernym_paths())
# 4개의 관계 그래프 경로
# 경로의 흐름에 따라 의미가 구체화됨

[[Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('causal_agent.n.01'), Synset('person.n.01'), Synset('adult.n.01'), Synset('man.n.01')], [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('living_thing.n.01'), Synset('organism.n.01'), Synset('person.n.01'), Synset('adult.n.01'), Synset('man.n.01')], [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('causal_agent.n.01'), Synset('person.n.01'), Synset('male.n.02'), Synset('man.n.01')], [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('living_thing.n.01'), Synset('organism.n.01'), Synset('person.n.01'), Synset('male.n.02'), Synset('man.n.01')]]


In [ ]:
girl = wn.synset('girl.n.01')
woman = wn.synset('woman.n.01')
boy = wn.synset('boy.n.01')
guy = wn.synset('guy.n.01')
print(man.path_similarity(girl))
print(man.path_similarity(woman))
print(man.path_similarity(man))
print(man.path_similarity(boy))
print(man.path_similarity(guy))
# 1에 가까울수록 유사도가 높음

0.25
0.3333333333333333
1.0
0.3333333333333333
0.5


In [ ]:
'''
import pandas as pd

def get_term_frequency(document, word_dict=None):
  if word_dict is None:
    word_dict = {}

  words = []
  for w in range(len(document)):
    words = document[w].split()

    for w in words:
      word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

  return pd.Series(word_dict).sort_values(ascending=False)
'''

'\nimport pandas as pd\n\ndef get_term_frequency(document, word_dict=None):\n  if word_dict is None:\n    word_dict = {}\n\n  words = []\n  for w in range(len(document)):\n    words = document[w].split()\n\n    for w in words:\n      word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])\n  \n  return pd.Series(word_dict).sort_values(ascending=False)\n'

In [ ]:
def get_document_frequency(documents):
  dicts = []
  vocab = set([])
  df = {}

  for d in documents:
    tf = get_term_frequency(d)
    dicts += [tf]
    vocab = vocab | set(tf.keys())

  for v in list(vocab):
    df[v] = 0
    for dict_d in dicts:
      if dict_d.get(v) is not None:
        df[v] += 1

  return pd.Series(df).sort_values(ascending=False)

In [ ]:
def get_tfidf(docs):
  vocab = {}
  tfs = []
  for d in docs:
    vocab = get_term_frequency(d, vocab)
    tfs += [get_term_frequency(d)]
  df = get_document_frequency(docs)

  from operator import itemgetter
  import numpy as np

  stats = []
  for word, freq in vocab.items():
    tfidfs = []
    for idx in range(len(docs)):
      if tfs[idx].get(word) is not None:
        tfidfs += [tfs[idx][word] * np.log(len(docs)/df[word])]
      else:
        tfidfs += [0]

    stats.append((word, freq, *tfidfs, max(tfidfs)))

  return pd.DataFrame(stats, columns=('word', 'frequency', 'doc1', 'doc2', 'doc3', 'max')).sort_values('frequency', ascending=False)

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 47.5 MB/s 


In [ ]:
import pandas as pd
from konlpy.tag import Hannanum
hannanum = Hannanum()

def get_term_frequency(document, word_dict=None):
  if word_dict is None:
    word_dict = {}

  words = []
  for w in range(len(document)):
    sub = hannanum.morphs(doc1[w])
    for i in range(len(sub)):
      words.append(sub[i])

  for w in words:
    word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

  return pd.Series(word_dict).sort_values(ascending=False)


doc1 = list()
with open('doc1.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r"\.\s|\?\s|\!\s|\n", line):
            if l: doc1.append(l)

In [ ]:
from google.colab import files
myfile = files.upload()

Saving doc1.txt to doc1 (1).txt


In [ ]:
from google.colab import files
myfile = files.upload()

Saving doc2.txt to doc2 (1).txt


In [ ]:
from google.colab import files
myfile = files.upload()

Saving doc3.txt to doc3 (1).txt


In [ ]:
import re

doc1 = list()
with open('doc1.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r"\.\s|\?\s|\!\s|\n", line):
            if l: doc1.append(l)

doc2 = list()
with open('doc2.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r"\.\s|\?\s|\!\s|\n", line):
            if l: doc2.append(l)

doc3 = list()
with open('doc3.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r"\.\s|\?\s|\!\s|\n", line):
            if l: doc3.append(l)


In [ ]:
words = []
for w in range(len(doc3)):
    sub = hannanum.morphs(doc3[w])
    for i in range(len(sub)):
      words.append(sub[i])

pre = ' '.join(words)
print(pre)

바 에 가 아 보 시ㄴ 적 있 으시죠 하 지만 바 에 가 ㄴ 덕분 에 2억 달러 짜리 계약 을 따 게 되 ㄴ 적 은 없 을 거 이 예요 10년 전 에 우리 에게 그러 ㄴ 일 이 있 었습니다 끔찍 하 ㄴ 하루 이 었던 어느 날 이 었어요 크 ㄴ 고객사가 우리 를 거의 죽이 려 들 었거든요 우리 는 소프트웨어 컨설팅 회사 이 었는데 특정 프로그래밍 스킬 이 없 어서 고객 이 원하 는 최첨단 클라우드 시스템 을 구축 하 ㄹ 수 없 었습니다 엔지니어 는 많 아지만 아무 도 그 고객 을 만족 시키 ㄹ 수 없 었습니다 우리 는 거의 잘리 ㄹ 지경 이 었죠 그래서 우리 는 바 에 가 아ㅂ니다 거기 서 바텐더 제프 와 이런저런 얘 이 기 를 나누 었죠 그 는 좋 은 바텐더 가 하 ㄹ 일 을 다 하 어 주 었습니다 우리 얘 이 기 에 공감 하 어 주 고 , 기분 을 풀 어 주 고 우리 고충 을 달래 기 도 하 었죠 " 그 고객들 이 너무 하 었네요 신경쓰 지 말 세 요 ." 그러 어다 그 는 진지 하 게 이렇 게 말 하 었습니다 " 저 를 그 고객사 로 보내 어 주세요 저 가 방법 을 찾 아 볼게요 ." 다음날 아침 , 우리 는 팀 회의 를 하 었습니다 물론 다들 술 이 덜 깨 ㄴ 상태 이 었죠 .. 저 는 반농담조 로 말 하 었습니다 뭐 어차피 잘리 ㄹ 판이 이 었으니까요 이렇 게 말 하 었죠 " 바텐더 제프 를 보내 면 어떨까요 ?" 잠시 침묵 이 흐르 고 다들 놀라 는 듯 하 더니 마침내 팀장 이 말 하 었습니다 " 좋 은 아이디어 이 군 요 ." "제프 는 매우 똑똑하 고 뛰 어 나 ㄴ 사람 이 죠 뭔가 방법 을 찾 을 거 ㅂ니다 그 를 고객 에게 보내 ㅂ시다 ." 제프 는 프로그래머 가 아니 ㅂ니다 . 사실 , 펜실베니아 대학 에서 철학 을 공부 하 어다 중퇴 하 었죠 하 지만 똑똑하 었습니다 문제 를 깊이 보 ㄹ 능력 이 있 었죠 우리 도 자 ㄹ 리 이 기 직전 이 라 그래서 그 를 보내 었습니다 긴장 의 몇 일 이 지나 ㄴ 후 제프 는 계속 고객사 에 있 었습니다

In [ ]:
import re

doc1 = list()
with open('doc1.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r" ", line):
            if l: doc1.append(l)

doc2 = list()
with open('doc2.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r" ", line):
            if l: doc2.append(l)

doc3 = list()
with open('doc3.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r" ", line):
            if l: doc3.append(l)


In [ ]:
word_tokens = doc1
stop_words = "ㄴ ㄹ ㅂ니다 을 를 는 이 가 있 습니다 의 과 ,"

result = []
for w in word_tokens:
  if w not in stop_words:
    result.append(w)

print(word_tokens,'\n')
print(result)


['오늘', '저는', '기술이', '어떤', '방향으로', '가고', '있는지에', '대해', '얘기해볼까', '합니다.', '흔히,', '기술을', '처음', '접할', '때', '우리는', '기술이', '가져다', '주는', '것에', '대해', '놀라곤', '합니다.', '하지만', '사실', '기술의', '대부분은', '예측이', '가능합니다.', '이는', '모든', '종류의', '기술', '시스템이', '각각의', '성향을', '가지기', '때문입니다.', '기술마다', '긴급한', '부분이', '있고', '일정한', '경향성을', '띄기', '마련입니다.', '그리고', '그런', '경향들은', '물리학의', '본질이나', '선과', '스위치', '간의', '화학', ',그리고', '전자등으로부터', '유래되고', '이는', '반복되는', '패턴을', '끊임없이', '만들어냅니다.', '그래서', '그', '패턴들은', '경향성과', '성향을', '만듭니다.', '여러분은', '이를', '중력과', '비슷하다고', '생각할', '수', '있습니다.', '계곡에', '빗방울', '떨어지는', '모습을', '상상해보세요.', '계곡을', '내려가는', '빗방울의', '실제', '경로는', '예측할', '수', '없습니다.', '우리는', '그들이', '어디로', '가는지', '알', '수', '없지만', '대략적인', '방향은', '필연적입니다.', '바로', '아래', '쪽이지요.', '이처럼', '기술', '시스템에', '내장된', '경향성과', '촉발성은', '우리에게', '사건이', '대략적으로', '어떻게', '진행되는지에', '대해', '알려줍니다.', '그래서', '크게', '봤을', '때', '전화기의', '발명은', '필연적이었다고', '봅니다.', '그러나', '아이폰은', '아니었죠.', '인터넷의', '발명도', '필연적이었지만', '트위터는', '아니었습니다.', '그래서', '현재', '여러', '흐름이', '존재하는데', 

In [ ]:
get_tfidf([doc1, doc2, doc3])

,word,frequency,doc1,doc2,doc3,max
0,이,532,0.000000,0.0,0.0,0.000000
1,하,374,0.000000,0.0,0.0,0.000000
2,는,255,0.000000,0.0,0.0,0.000000
3,ㅂ니다,235,0.000000,0.0,0.0,0.000000
4,을,231,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...
698,투자,1,1.098612,0.0,0.0,1.098612
697,비교,1,1.098612,0.0,0.0,1.098612
696,솔직히,1,1.098612,0.0,0.0,1.098612
695,정말,1,1.098612,0.0,0.0,1.098612


In [ ]:
def get_tf(docs):
  vocab = {}
  tfs = []
  for d in docs:
    vocab = get_term_frequency(d, vocab)
    tfs += [get_term_frequency(d)]

  from operator import itemgetter
  import numpy as np

  stats = []
  for word, freq, in vocab.items():
    tf_v = []
    for idx in range(len(docs)):
      if tfs[idx].get(word) is not None:
        tf_v += [tfs[idx][word]]
      else:
        tf_v += [0]

    stats.append((word, freq, *tf_v))

  return pd.DataFrame(stats, columns=('word', 'frequency', 'doc1', 'doc2', 'doc3')).sort_values('frequency', ascending=False)

In [ ]:
get_tf([doc1,doc2,doc3])

{'오늘': 1, '저는': 5, '기술이': 2, '어떤': 1, '방향으로': 1, '가고': 1, '있는지에': 2, '대해': 6, '얘기해볼까': 1, '합니다': 8, '흔히,': 1, '기술을': 1, '처음': 1, '접할': 1, '때': 5, '우리는': 19, '가져다': 1, '주는': 1, '것에': 6, '놀라곤': 1, '하지만': 6, '사실': 4, '기술의': 1, '대부분은': 1, '예측이': 1, '가능합니다': 1, '이는': 6, '모든': 7, '종류의': 6, '기술': 2, '시스템이': 1, '각각의': 1, '성향을': 2, '가지기': 1, '때문입니다': 3, '기술마다': 1, '긴급한': 1, '부분이': 2, '있고': 5, '일정한': 1, '경향성을': 1, '띄기': 1, '마련입니다': 1, '그리고': 12, '그런': 2, '경향들은': 1, '물리학의': 1, '본질이나': 1, '선과': 1, '스위치': 1, '간의': 1, '화학': 1, ',그리고': 1, '전자등으로부터': 1, '유래되고': 1, '반복되는': 1, '패턴을': 1, '끊임없이': 2, '만들어냅니다': 1, '그래서': 11, '그': 9, '패턴들은': 1, '경향성과': 2, '만듭니다': 1, '여러분은': 6, '이를': 5, '중력과': 1, '비슷하다고': 1, '생각할': 1, '수': 20, '있습니다': 18, '계곡에': 1, '빗방울': 1, '떨어지는': 1, '모습을': 1, '상상해보세요': 1, '계곡을': 1, '내려가는': 1, '빗방울의': 1, '실제': 2, '경로는': 1, '예측할': 1, '없습니다': 4, '그들이': 5, '어디로': 1, '가는지': 1, '알': 1, '없지만': 1, '대략적인': 1, '방향은': 1, '필연적입니다': 1, '바로': 13, '아래': 1, '쪽이지요': 1, '이처럼': 3, '시스템에': 1, '내장된': 1, '촉발성은'

,word,frequency,doc1,doc2,doc3
0,수,44,20,15,9
1,있습니다,41,18,17,6
2,것입니다,40,12,22,6
3,우리는,39,19,12,8
4,우리가,32,23,6,3
...,...,...,...,...,...
1100,동등하게,1,0,0,1
1099,취급되어야,1,0,0,1
1098,여러분도,1,0,0,1
1097,예술가를,1,0,0,1


In [ ]:
import torch

def get_l1_distance(x1, x2):
  return ((x1-x2).abs()).sum()

def get_l2_distance(x1, x2):
  return ((x1-x2)**2).sum()**.5

def get_infinity_distance(x1, x2):
  return ((x1-x2).abs()).max()

def get_cosine_similarity(x1, x2):
  return (x1*x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

def get_jaccard_similarity(x1, x2):
  return torch.stack([x1,x2]).min(dim=0)[0].sum() / torch.stack([x1,x2]).max(dim=0)[0].sum()


In [ ]:
# konlpy
import konlpy
from konlpy.tag import Hannanum

hannanum = Hannanum()

doc1 = list()
with open('doc1.txt', encoding='UTF8') as file:
    for line in file:
        for l in re.split(r"\.\s|\?\s|\!\s|\n", line):
            if l: doc1.append(l)

sentence = []
for i in range(len(doc1)):
  words = hannanum.morphs(doc1[i])
  sentence.append(' '.join(words))

print(sentence)

['오늘 저 는 기술 이 어떤 방향 으로 가 고 있 는 지 에 대 어 하 어 얘기 하 어 보 ㄹ까 하 ㅂ니다', '흔히, 기술 을 처음 접하 ㄹ 때 우리 는 기술 이 가 아 지 어다 주 는 것 에 대 어 하 어 놀라 곤 하 ㅂ니다', '하 지만 사 시ㄹ 기술 의 대부분 은 예측 이 가능 하 ㅂ니다', '이 는 모든 종류 의 기술 시스템 이 각각 의 성향 을 가 아 지 기 때문 이 ㅂ니다', '기술 마다 긴급 하 ㄴ 부분 이 있 고 일정한 경향성 을 띄 기 마련 이 ㅂ니다', '그리고 그러 ㄴ 경향들 은 물리학 의 본질 이나 선 과 스위치 간 의 화학 , 그 이 리 고 전자등 으로부터 유래 되 고 이 는 반복 되 는 패턴 을 끊임없이 만들 어 내 ㅂ니다', '그래서 그 패턴들 은 경향성 과 성향 을 만들 ㅂ니다', '여러분 은 이 를 중력 과 비슷 하 다 고 생각 하 ㄹ 수 있 습니다', '계곡 에 빗방울 떨 어 지 는 모습 을 상상 하 어 보 세요', '계곡 을 내리 어 가 는 빗방울 의 실제 경로 는 예측 하 ㄹ 수 없 습니다', '우리 는 그 들 이 어디 로 가 는지 알 ㄹ 수 없 지 말 ㄴ 대략적 이 ㄴ 방향 은 필연적 이 ㅂ니다', '바로 아래 쪽 이 지요', '이 처럼 기술 시스템 에 내장 되 ㄴ 경향성 과 촉발성 은 우리 에게 사건 이 대략적 으로 어떻 게 진행 되 는 지 에 대 어 하 어 알 려 줄 ㅂ니다', '그래서 크 게 보 아ㄹ 때 전화기 의 발명 은 필연적 이 었다 고 보 ㅂ니다', '그러나 아이폰 은 아니 었죠', '인터넷 의 발명 도 필연적 이 었지만 트위터 는 아니 었습니다', '그래서 현재 여러 흐르 ㅁ 이 존재 하 ㄴ데 저 는 그 중 에 가장 중요 하 ㄴ 것 이 끊임없이 더욱 더 지능적 으 로 고도화 되 는 경향성 이 ㅂ니다', "그것 을 ' cognification ' 이르 어 하 ㅂ니다", '저 가 붙이 ㄴ 거 ㅂ니다', '이 것 은 인공지능 혹은 AI 라고도 불 리ㅂ니다', '그리고 저 는 이것 이 앞 으로 20년 동안 우리 

In [ ]:
from collections import defaultdict
import pandas as pd

def get_context_counts(lines, w_size=2):
  co_dict = defaultdict(int)

  for line in lines:
    words = line.split()

    for i, w in enumerate(words):
      for c in words[i - w_size:i + w_size]:
        if w != c:
          co_dict[(w,c)] += 1

  return pd.Series(co_dict)

In [ ]:
def co_occurrence(co_dict, vocab):
  data = []
  for word1 in vocab:
    row = []

    for word2 in vocab:
      try:
        count = co_dict[(word1, word2)]
      except KeyError:
        count = 0
      row.append(count)

    data.append(row)

  return pd.DataFrame(data, index=vocab, columns=vocab)

In [ ]:
get_context_counts(sentence)

는    오늘    1
     저     3
     기술    2
기술   저     1
     는     2
          ..
뜻    하     1
하    뜻     1
ㅂ니다  뜻     1
     감사    1
.    하     1
Length: 5255, dtype: int64

In [ ]:
tfs = get_term_frequency(' '.join(sentence))

IndexError: ignored

In [ ]:
! pip install fasttext

     |████████████████████████████████| 68 kB 3.3 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3148173 sha256=2558ac176096c443f05d163367697d9e636f99a5dbc0cf035562643d6fae483a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
! fasttext skipgram -input ko.tok.txt -output do.tok -dim 256 -epoch 100 -minCount 5

/bin/bash: fasttext: command not found


In [ ]:
data = [['오늘', '기술', '방향', '얘기', '기술', '처음', '때', '기술', '것', '사실', '기술', '대부분', '예측', '종류', '기술', '시스템', '각각', '성향', '때문', '기술', '긴급', '부분', '경향', '마련', '경향', '물리학', '본질', '선', '스위치', '간', '화학', '전자', '등', '유래', '반복', '패턴', '패턴', '경향', '성향', '중력', '생각', '수', '계곡', '빗방울', '모습', '상상', '계곡', '빗방울', '실제', '경로', '예측', '수', '수', '대략', '방향', '필연', '아래', '쪽', '기술', '시스템', '내장', '경향', '촉발', '사건', '대략', '진행', '때', '전화기', '발명', '필연', '아이폰', '인터넷', '발명', '필연', '트위터', '흐름', '존재', '중', '것', '지능', '고도', '경향', '제가', '겁', '니다', '것', '인공지능', '라고', '앞', '년', '동안', '사회', '영향력', '발전', '유행', '방향', '추', '동력', '생각', '영향', '인공지능', '곳', '일', '병원', '사무실', '뒷편', '선', '의사', '진단', '법률', '사무소', '법', '증거', '검토', '인간', '법률가', '수행', '비행기', '사용', '실제', '조종사', '사실', '분', '분', '정도', '밖', '운전', '외', '운행', '주지', '넷플릭스', '아마존', '에선', '뒤', '활동', '추천', '오늘날', '대표', '예시', '세계', '최고', '바둑', '선수', '알파고', '다', '비디오 게임', '인공지능', '상대로', '게임', '겁', '니다', '최근', '구글', '인공지능', '비디오 게임', '법', '말하자면', '비디오 게임', '자체', '건', '기', '성취', '비디오 게임', '법', '것', '한걸음', '것', '인공', '총명', '것', '인공', '총명', '것', '요즘', '트렌드', '가지', '부분', '소평', '생각', '가지', '이해', '인공지능', '이해', '것', '인공지능', '을', '수용', '도움', '줄', '것', '인공지능', '진정', '조종', '수', '추세', '세부', '측면', '조종', '수', '것', '때문', '가지', '측면', '설명', '번', '지능', '지능', '인지', '이해', '점', '지능', '것', '1차원', '생각', '음표', '소리', '것', '생각', '대표', '예시', '검사', '쥐', '아이큐', '시작', '침팬지', '것', '바보', '것', '추측', '평균', '사람', '다음', '다음', '천재', '겁', '니다', '단수', '지능', '복수', '증가', '것', '사실', '관념', '지능', '인간', '지능', '지능', '단일', '음계', '인지', '악기', '연주', '음표', '교향곡', '안', '유형', '지능', '존재', '연역', '추리', '능력', '정서', '지능', '공간', '감각', '등', '안', '가지', '유형', '지능', '사람', '장점', '동물', '경우', '마찬가지', '그릇', '지능', '내용', '종류', '지능', '교향곡', '것', '악기', '것', '방식', '생각', '방식', '배열', '수', '특정', '부분', '사람', '수', '예', '다람쥐', '장기', '기억력', '천재', '먹이', '기억', '부분', '사람', '낮', '수', '기계', '때', '기계', '방법', '제작', '차원', '지능', '지능', '수준', '대부분', '인간', '필요', '때문', '것', '염두', '인공', '클러스터', '고려', '인공지능', '종류', '인', '지력', '추가', '구체', '계산기', '산수', '능력', '길', '구글', '장기', '기억', '생각', '유형', '적용', '수', '예', '자동차', '인공지능', '차', '안', '차', '운전', '이유', '차', '인간', '운전', '점', '때문', '생각', '차', '특징', '것', '정신', '분산', '것', '가스레인지', '금융', '전공', '등', '고민', '운전', '운전', '만', '특징', '광고', '수도', '생각', '식', '자각', '종류', '걱정', '것', '정신', '대부분', '경우', '것', '수', '생각', '형태', '겁', '니다', '형태', '종류', '생각', '영역', '데이터', '사실', '문제', '과학', '시장', '현실', '인간', '사고력', '수', '수', '단계', '프로그램', '필요', '것', '종류', '생각', '창조', '문제', '것', '예', '암흑물질', '양자중력', '작업', '외계', '지', '성체', '것', '바가', '일', '외계인', '수도', '사고', '수', '것', '생각', '창조', '엔진', '이자', '부', '경제', '시작', '번', '측면', '차', '산업', '혁명', '발생', '인공지능', '사용', '것', '제1차 산업', '혁명', '인공', '힘', '거', '사실', '기반', '이전', '농업 혁명', '동안', '것', '사람', '노동력', '동물', '힘', '목적', '인간', '동물', '육체', '노동', '산업', '혁명', '동안', '혁신', '변화', '증기', '기관', '화석', '연료', '사용', '인간', '것', '수', '인공', '힘', '겁', '니다', '오늘날', '고속도로', '주행', '스위치', '것', '마리', '말', '지휘', '효과', '마력', '고층 건물', '도시', '도로', '수', '의자', '냉장고', '공장', '인간', '한참', '힘', '인공', '힘', '집', '공장', '농가', '등', '그리드', '분배', '인공', '힘', '플러그', '수', '혁신', '근원', '농부', '수동', '식', '펌프', '인공', '힘', '전기', '적용', '자동', '식', '펌프', '수', '경우', '번', '반복', '공식', '산업', '혁명', '것', '과정', '사실', '인공지능', '경우', '에도', '적용', '수', '배', '전망', '인공지능', '분배', '인공', '힘', '자동', '펌프', '인공', '지능', '수', '스마트', '펌프', '수', '수', '반복', '차', '산업', '혁명', '것', '고속도로', '마력', '개', '생각', '자동', '주행', '자동차', '이건', '상품', '용도', '인공지능', '상공', '길', '전기', '것', '전기', '작동', '것', '인공지능', '것', '예언', '미래', '스타트', '업', '사업', '시작', '공식', '변수', '인공지능', '것', '공식', '일', '차', '산업', '혁명', '수', '당장', '구글', '로그인', '인공지능', '센트', '번', '수', '당장', '거란', '말', '번', '측면', '인공지능', '구체', '로봇', '때', '로봇', '수족', '것', '일', '직업', '일', '뭉텅이', '때문', '로봇', '직업', '개념', '정립', '겁', '니다', '일', '것', '때문', '한편', '로봇', '전', '일', '종류', '업무', '것', '로봇', '탄생', '직업', '군', '인간', '달성', '과제', '것', '자동화', '기기', '과거', '필요', '조차', '것', '이제', '수', '것', '로봇', '차지', '직업', '직업', '양상', '것', '점', '할당', '업무', '대부분', '효율', '생산성', '가치', '정의', '수', '것', '일', '구체', '설명', '수', '육체', '노동', '이든', '정신', '노동', '이든', '일', '효율', '생산성', '관점', '로봇', '대신', '수', '것', '분류', '수', '생산성', '로봇', '몫', '겁', '니다', '로봇', '건', '시간', '낭비', '것', '효율', '과학', '본질', '효율', '과학', '번', '실패', '점', '기반', '성공', '실험', '시험', '사실', '근거', '수', '과학', '효율', '부족', '점', '기반', '혁신', '정의', '효율', '겁', '니다', '시제품', '것', '실패', '작동', '언가', '시도', '모험', '근본', '효율', '예술', '효율', '인간관계', '효율', '것', '자연', '일', '효율', '때문', '효율', '인간', '로봇', '것', '앞', '인공지능', '일', '것', '생각', '블루', '세계', '제일', '체스', '챔피언', '때', '사람', '체스', '끝', '생각', '요즘', '세계', '제일', '체스', '챔피언', '인공지능', '인간도', '인공지능', '인간', '팀', '제일', '의료', '분석가', '의사', '인공지능', '의사', '인공지능', '팀', '앞', '인공지능', '일', '겁', '니다', '미래', '로봇', '일', '수', '연봉', '결정', '것', '라고', '생각', '세번', '포인트', '것', '도구', '예', '심', '게', '존재', '거부', '일', '미래', '미래', '년', '후', '지금', '때', '인공지능', '이해', '말', '생각', '말하자면', '건', '앞', '년', '간', '것', '인터넷', '정도', '안', '당장', '인공지능', '전문가', '돈', '인공지능', '달러', '쯤', '투자', '사업', '년', '후', '비교', '전문가', '만', '초기', '단계', '것', '시작', '점', '것', '인터넷', '시작', '점', '앞', '것', '시작', '점', '년', '후', '모두', '사용', '영향력', '인공지능', '발명', '뜻', '감사']
,['자리', '크리에이터', '디자이너', '엔지니어', '기업가', '예술가', '상상력', '분이', '손', '전해', '드릴', '소식', '지금', '년', '동안', '2000년', '간', '인간', '일', '방식', '변화', '변화', '것', '인류사', '시대', '새벽', '인간', '일', '방식', '시대', '수', '수렵', '채집', '시대', '년', '간', '지속', '다음', '농경', '시대', '년', '간', '지속', '산업', '시대', '수', '기간', '정보', '시대', '년', '간', '지속', '오늘날', '종', '다음', '시대', '시작', '증강', '시대', '오신', '것', '환영', '시대', '인간', '사고', '컴퓨터', '시스템', '인간', '자연', '능력', '증강', '겁', '니다', '로봇', '시스템', '제작', '디지털', '경계', '자연', '감각', '너머', '세계', '연결', '것', '인식', '증강', '시작', '여러분', '중', '분이', '증강', '사이보그', '인간', '증강', '주장', '테', '파티', '상상', '누', '군가', '답', '질문', '때', '초', '만', '답', '수', '시작', '뿐', '조차', '수동', '도구', '사실', '년', '동안', '인류', '사용', '도구', '완벽', '수', '동적', '인간', '지시', '아무것', '최초', '도구', '인간', '지점', '예술가', '의도', '조각', '진보', '도구', '조차', '인간', '지시', '아무것', '사실', '오늘날', '문자', '인간', '손', '컴퓨터', '의도', '한계', '것', '실망감', '스타 트렉', '컴퓨터', '대화', '자동차', '디자인', '컴퓨터', '말', '컴퓨터', '자동차', '독일제', '컴퓨터', '게', '선택', '지', '제시', '것', '위', '언급', '대화', '분', '생각', '거리', '것', '연구', '것', '도구', '생산', '것', '변화', '중', '생산', '디자인', '도구', '기하학', '구조', '컴퓨터', '알고리즘', '사용', '디자인', '목표', '제약', '예', '무인 항공기', '차', '대', '경우', '말', '개', '프로펠러', '공기역학', '효율', '것', '컴퓨터', '해결', '방법', '탐색', '개', '해법', '기준', '것', '작업', '컴퓨터', '담당', '결과', '상상', '디자인', '컴퓨터', '제안', '것', '아무도', '적', '무', '시작', '것', '무인 항공기', '몸체', '날', '다람쥐', '골반', '것', '우연이', '진화', '방식', '작동', '알고리즘', '설계', '때문', '현실', '기술', '시작', '것', '일', '년', '간', '협력', '미래', '지향', '비행기', '연구', '중', '제자리걸음', '최근', '인공지능', '이용', '다음', '결과', '컴퓨터', '디자인', '것', '3D', '프린터', '출력', '한', '칸막이', '기존', '것', '무게', '절반', '올해', '말', '쯤', '320', '탑재', '예정', '컴퓨터', '정의', '문제', '해법', '수', '직관', '컴퓨터', '학습', '적', '무', '작업', '제일', '진보', '도구', '의미', '주인', '목줄', '산책', '시간', '것', '것', '목줄', '때', '산책', '때문', '것', '가지', '집중', '일', '기억', '기억', '유지', '머릿속', '패턴', '흥미', '과정', '컴퓨터 과학자', '년', '오랫동안', '인공', '지능', '적용', '것', '일치', '년', '과학자', '틱택토', '게임', '수', '컴퓨터', '설계', '그 후', '년', '년', '체스', '년', '컴퓨터', '체스', '사람', '사실', '설정', '방안', '사용', '이', '인간', '추론', '전', '세계', '제일', '게임', '중', '바둑', '세계', '최고', '수', '사실', '바둑', '우주', '원자', '경우의 수', '직관', '개발', '사실', '일부', '수', '프로그래머', '이해', '것', '사람', '생애', '시간', '안', '컴퓨터', '아이들', '게임', '전략', '사고', '정점', '인식', '상황', '일', '것', '상태', '컴퓨터', '상황', '수', '순수', '논리', '직관', '것', '다리', '대부분', '결정', '것', '순간', '것', '다리', '위험', '것', '개발', '시작', '시스템', '직관', '일종', '시일', '내', '디자인', '것', '컴퓨터', '말', '것', '작동', '노래', '사람', '아이스크림', '향도', '요청', '수', '겁', '니다', '일', '이전', '문제', '해결', '컴퓨터', '작업', '수', '예', '기후 변화', '문제', '인간', '힘', '도움', '수', '것', '총동원', '수', '제가', '말', '것', '기술', '인', '지력', '증폭', '수', '기술', '시대', '사람', '것', '상상', '디자인', '수', '인간', '고안', '디자인', '것', '인간', '증강', '시대', '가상', '지적', '영역', '현실', '세계', '관련', '생각', '기술', '인간', '증강', '현실', '세계', '로봇', '시스템', '로봇', '일자리', '것', '분야', '것', '흥미', '점', '인간', '로봇', '작업', '것', '서로', '증강', '세상', '시작', '것', '샌프란시스코', '응용', '연구소', '관심', '분야', '중', '차세대', '로봇', '인간', '로봇', '협업', '이건', '로봇', '중', '실험', '공사장', '벽', '콘센트', '전등', '스위치', '구멍', '것', '반복', '일', '사람', '인간', '파트너', '개', '것', '영어', '몸짓', '지시', '지시', '완벽', '수행', '인간', '이용', '인간', '장점', '인식', '지각', '의사', '결정', '로봇', '이용', '로봇', '장점', '정밀도', '반복', '수행', '프로젝트', '프로젝트', '목표', '인간', '컴퓨터', '로봇', '문제', '시범', '경험', '것', '인간', '노동자', '역할', '공사', '현장', '대나무', '대나무', '형태', '로봇', '로봇', '섬유', '작업', '인간', '일', '것', '통제', '인공지능', '사용', '인공지능', '인간', '로봇', '계속', '개', '개별', '구성 요소', '추적', '흥미', '점', '임시', '구조물', '일', '인간', '로봇', '인공지능', '서로', '증강', '것', '짓', '프로젝트', '활동', '예술가', '팀', '3D', '디자인', '로봇', '출력', '세계', '최초', '다리', '작업', '말씀', '대로', '인공', '지능', '디자인', '작업', '완료', '버튼', '로봇', '스테이', '리스', '스틸', '3D', '시작', '것', '인간', '개입', '다리', '완성', '때', '출력', '계속', '겁', '니다', '컴퓨터', '것', '상상', '디자인', '능력', '증강', '것', '로봇', '시스템', '전', '인간', '수', '것', '텐', '데', '인간', '것', '판단', '제어', '능력', '가요', '것', '경계', '경계', '인간', '경계', '주변', '것', '파악', '것', '경계', '시작', '예', '자동차', '도시', '공공', '사업', '국', '길', '모퉁이', '도로', '팬', '곳', '말', '건물', '디자이너', '안', '사람', '말', '하진', '장난감', '제조', '업체', '장난감', '대상', '디자이너', '바비', '인형', '디자인', '때', '삶', '상상', '바비', '인형', '현실', '디자이너', '현실', '디자인', '도로', '건물', '바비', '일', '정보', '사용자', '것', '사용', '수', '부족', '부분', '신경', '디자인', '사용', '것', '연결', '신경', '만일', '여러분', '모두', '것', '정보', '것', '시간', '에너지', '사용', '작년', '달러', '제품', '소비자', '설득', '사용', '만일', '디자인', '것', '현실', '세계', '출시', '후', '신경', '시스템', '연결', '후', '출시', '시점', '수', '제품', '소비자', '것', '처음', '소비자', '것', '것', '희소식', '개발', '중', '디지털', '신경', '계가', '디자인', '것', '연결', '겁', '니다', '형제', '팀', '프로젝트', '진행', '중', '것', '중', '하나로', '광란', '질주', '차', '제정신', '최선', '진행', '일', '전통', '경주', '차', '차', '신경', '제공', '것', '개', '센서', '세계', '수준', '운전자', '사막', '일주일', '간', '자동차', '경계', '자동차', '일', '저장', '차', '해진', '포함', '개', '데이터', '그 후', '정신', '짓', '데이터', '디자인', '인공', '지능', '우리는 신', '경계', '디자인', '도구', '최고', '차', '요청', '수', '이전', '인간', '번', '디자인', '것', '인간', '디자인', '제외', '디자인', '인공', '지능', '증강', '인간', '겁', '니다', '디지털', '경계', '로봇', '것', '제작', '수', '만일', '미래', '증강', '시대', '인지', '육체', '지각', '증강', '것', '세상', '생각', '물건', '제작', '것', '물건', '것', '세상', '것', '건설', '것', '성장', '것', '것', '고립', '것', '연결', '것', '이동', '것', '추출', '것', '축적', '방향', '겁', '니다', '주변', '사물', '지시', '자율', '가치', '쪽', '것', '인간', '능력', '증강', '것', '감사', '세상', '극', '변화', '겁', '니다', '세상', '연결', '역동', '것', '수용', '것', '것', '앞', '모습', '이전', '것', '것', '세상', '모습', '기술', '자연', '인간', '사이', '협력', '관계', '덕분', '기대', '만', '가치', '미래']
,['바', '적', '바', '간', '덕분', '달러', '계약', '거', '예', '년', '전', '일', '하루', '날', '고객', '사가', '소프트웨어', '컨설팅', '회사', '특정', '프로그래밍', '스키', '이', '고객', '첨단', '클라우드', '시스템', '구축', '수', '엔지니어', '아무도', '고객', '만족', '수', '지경', '바', '바텐더', '제프', '얘기', '바텐더', '일', '얘기', '공감', '기분', '고충', '고객', '신경', '말', '고객', '사', '제가', '방법', '다음날', '아침', '팀', '회의', '다', '술', '상태', '농담', '조로', '말', '판', '말', '바텐더', '제프', '침묵', '다', '듯', '팀장', '말', '아이디어', '제프', '사람', '방법', '겁', '니다', '고객', '제프', '프로그래머', '사실', '펜', '실', '베니', '대학', '철학', '공부', '중퇴', '문제', '능력', '직전', '긴장', '일', '후', '제프', '고객', '사', '고객', '수', '내막', '고객', '프로그래밍', '스키', '집착', '논의', '주제', '대상', '무어', '목적', '논의', '주제가', '제프', '해결책', '프로그래밍', '방법', '고객', '사례', '중', '당시', '명', '직원', '절반', '컴퓨터 과학', '공학', '전공자', '제프', '사례', '한가지', '의문점', '업무', '적용', '수', '채용', '교육', '방식', '컴퓨터 과학', '공학', '전공자', '예술가', '음악가', '작가', '포함', '제프', '경우', '반복', '시작', '최고', '기술', '책임자', '영어', '전공자', '맨해튼', '자전거', '택배', '현재', '직원', '명', '명', '이하', '컴퓨터 과학', '공학', '전공자', '컴퓨터', '컨설팅', '회사', '시장', '일등', '기업', '급', '성장', '소프트웨어 패키지', '연', '매출', '달러', '효과', '거', '반면', '우리나라', '기반', '교육', '강요', '과학', '기술', '공학', '수학', '모두', '모습', '실수', '년', '이후', '미국', '전공자', '증가', '반면', '인문학', '공자 수', '대통령', '10억', '달러', '이상', '교육', '투자', '과목', '예산', '말', '대통령', '최근', '교육부', '예산', '중', '달러', '컴퓨터 과학', '쪽', '투자', '기업', '공학', '전공', '인력', '부족', '불평', '움직임', '기술', '경제', '성공', '말', '시가총액', '세계', '10대', '기업', '중', '개', '기술', '관련', '회사', '예측', '것', '미래', '인력', '경력', '것', '이론', '매력', '과대평가', '축구팀', '선수', '모두', '공', '한곳', '구석', '모양새', '공', '가치', '부여', '안', '이상', '인문학', '이상', '과학', '가치', '부여', '안', '가지', '이유', '오늘날', '기술', '직관', '전공자', '채용', '수', '전문', '분야', '눈', '수', '이유', '최신', '시스템', '프로그래밍', '기술', '수', '때문', '레고', '조립', '프로그래밍', '학습', '필요', '광범위', '정보', '때문', '인력', '전문', '능력', '필요', '능력', '정형', '교육', '과거', '말', '필요', '차별', '능력', '직관', '기술', '세계', '인간', '일', '데', '도움', '능력', '일', '것', '최종', '산출물', '구상', '것', '효용', '성도', '포함', '실제', '세계', '경험', '판단', '역사', '맥락', '요구', '제프', '사례', '것', '고객', '잘못', '곳', '집중', '것', '전형', '형태', '기술자', '기업', '최종 사용자', '소통', '기업', '자신', '요구', '사항', '설명', '경우', '표면', '인간', '소통', '개발', '능력', '말', '과학', '언가', '반면', '인문학', '제가', '불편', '때', '사람', '인문학', '경력', '취급', '때', '여러분', '인문학', '세상', '맥락', '제공', '비판', '사고', '방법', '인문학', '의도', '구조', '반면', '과학', '의도', '구조', '인문학', '설득', '언어', '제공', '감성', '사고', '행동', '전환', '인문학', '과학', '동등', '취급', '예술가', '채용', '기술', '회사', '설립', '수', '결과', '것', '오늘', '이', '말', '것', '여성', '프로그래머', '반대', '것', '제가', '운전', '다음', '다리', '다음', '엘리베이터', '이면', '공학', '편집증', '미래', '직업', '이', '지배', '것', '것', '친구', '자녀', '친척', '손주', '여자', '조카', '남자', '조카', '대로', '수', '격려', '일자리', '것', '기술', '졸업장', '요구', '채용', '아', '세', '구글', '애플', '페이스북', '회사', '일자리', '중', '기술직', '판', '매직', '디자이너', '프로젝트 매니저', '프로그램', '매니저', '제품', '매니저', '변호사', '인사', '관리', '전문가', '강사', '코치', '영업', '구매', '담당자', '등', '일', '사람', '미래', '인력', '필요', '한가지', '다', '동의', '생각', '다양성', '다양성', '성별', '인종', '끝', '나선', '안', '필요', '건', '배경', '다양성', '능력', '다양성', '내성', '사람', '외향', '사람', '사람', '사람', '미래', '인력', '기술', '인력', '자유', '자신', '것']]

In [ ]:
from gensim.models import FastText

embedding = FastText(data, size=100, window=7, negative=3, min_count=5)
embedding.save('fasttext.model')

model = FastText.load('fasttext.model')
print(model.wv['인공지능'])
print(model.most_similar('인공지능'))

[-7.2016024e-05 -1.8346038e-03  2.3303700e-03 -5.9494638e-04
  2.5070668e-03  2.5580435e-03  3.0331768e-04  5.3263165e-04
 -8.2824973e-04 -2.8264979e-03 -3.2569885e-03  9.2922745e-04
  1.4796389e-03  1.7676638e-03  9.0155570e-04 -2.2840744e-03
  3.8270636e-03 -3.1362992e-04  2.5162732e-03  2.7900346e-04
 -2.9032230e-03  1.9817122e-03 -2.1441376e-03 -5.1172995e-03
  8.1473372e-05  1.2570281e-04 -2.3611265e-03  2.5320272e-03
  2.5138403e-03 -2.5730403e-03  4.0781029e-04 -3.0369752e-05
  2.1145025e-03  8.5310318e-04  7.8549556e-04 -7.2389434e-04
  1.1848909e-03 -4.9776153e-04 -1.7528980e-03 -2.3660744e-03
  3.5379634e-03  3.2914251e-03  7.8175093e-05  4.6666544e-03
 -2.9600568e-03  3.0545748e-03  3.3748366e-03  2.5703248e-03
 -3.5720682e-04 -2.2432420e-03 -2.4262141e-03  4.4297557e-03
 -1.4584485e-03 -7.6619000e-04  6.8215490e-04  4.4034544e-04
 -1.3048560e-03 -3.1120456e-03  2.3790567e-04 -8.5192063e-04
  8.2493381e-04  2.5649377e-04  1.8780296e-03 -2.9221716e-04
  1.9550703e-03 -9.23926

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
